In [1]:
from run import run, getOutputFileName
import os 
from hydra import compose, initialize
from omegaconf import OmegaConf
import logging 
import json
import pickle

def run_all_experiments_for_attack(attack_name: str, output_dir: str, attack_ratio=0.25):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created directory: {output_dir}")
    results_path = output_dir + attack_name + "/"
    if not os.path.exists(results_path): 
        os.makedirs(results_path)
        print(f"Created directory {results_path}")
    logging.basicConfig(filename=output_dir+"experiment_errors.log", level=logging.DEBUG, format='%(asctime)s - %(levelname)s: %(message)s')
    new_configs_path =output_dir+"runconfigs/"
    if os.path.exists(new_configs_path):
        os.rmdir(new_configs_path)
    os.makedirs(new_configs_path)

    for model_file in ["rnn"]: #"base", "mlp", "lgr", "lsvc", "xgb_bagging", "rf", "lstm", 
        with initialize(version_base=None, config_path="conf", job_name="test_app"):
            cfg = compose(config_name=model_file)
        num_clients_per_model = {
            "SCNN": 100, 
            "MLP": 100,
            "LGR": 100, 
            "LSVC": 100, 
            "LSTM": 50,
            "RNN": 50, 
            "RF": 50,
            "XGB": 50 
        }
        experiment_config = [("FULL", attack_ratio), ("END", attack_ratio)] #("FULL", 0), , ("MID", attack_ratio), 
        model = cfg.model
        print("Running", attack_name, "on", model)
        cfg.num_clients, cfg.num_clients_per_round_fit = num_clients_per_model[model], num_clients_per_model[model]
        cfg.attack_type=attack_name
        for (attack_round, ratio) in experiment_config: 
            cfg.max_attack_ratio = ratio 
            cfg.attack_round = attack_round
            try:
                config_file_name = f"{model}_{attack_name}_{attack_round}_{ratio*100}"
                OmegaConf.save(config=cfg, f=new_configs_path+config_file_name+".yaml") 
                results = run(config_path=new_configs_path, config_name=config_file_name)
                with open(results_path+getOutputFileName(model=model, attack_round=attack_round, attack_ratio=ratio), "wb") as h: 
                    pickle.dump(results, h, protocol=pickle.HIGHEST_PROTOCOL)
                    
            except Exception as e: 
                error_message = {"attack_name":attack_name, "model":model, "error":str(e)}
                error_message = json.dumps(error_message)
                logging.error(error_message)
                logging.log
    for handler in logging.root.handlers[:]:
        handler.close()

In [2]:
run_all_experiments_for_attack(attack_name="LF", output_dir="runs/")

Created directory: runs/
Created directory runs/LF/


c:\Users\rohit\Documents\workspace\projects\federated\run.py:16: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path=config_path):
INFO flwr 2024-04-08 17:20:03,700 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


Running LF on RNN
model: RNN
num_rounds: 10
num_clients: 50
num_classes: 10
num_clients_per_round_fit: 50
num_clients_per_round_eval: 10
max_attack_ratio: 0.25
attack_round: FULL
attack_type: LF
config_fit:
  lr: 0.1
  local_epochs: 10
  batch_size: 20



2024-04-08 17:20:08,545	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-04-08 17:20:11,004 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 1603815015.0, 'object_store_memory': 801907507.0, 'node:127.0.0.1': 1.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'CPU': 12.0}
INFO flwr 2024-04-08 17:20:11,004 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-04-08 17:20:11,005 | app.py:242 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0.5}
INFO flwr 2024-04-08 17:20:11,019 | app.py:288 | Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO flwr 2024-04-08 17:20:11,020 | server.py:89 | Initializing global parameters
INFO flwr 2024-04-08 17:20:11,021 | server.py:276 | Requesting initial parameters from one random client


KeyboardInterrupt: 